In [ ]:
#| default_exp evaluation

# Hierarchical Evaluation

To assist the evaluation of hierarchical forecasting systems, we make available accuracy metrics along with the `HierarchicalEvaluation` module that facilitates the measurement of prediction's accuracy through the hierarchy levels. 

The available metrics include point and probabilistic multivariate scoring rules that were used in previous hierarchical forecasting studies.

In [ ]:
#| export
import narwhals as nw
import numpy as np

from hierarchicalforecast.utils import _to_narwhals_maybe_warn_and_reset_idx, _to_native_maybe_set_index
from inspect import signature
from narwhals.typing import Frame
from scipy.stats import multivariate_normal
from typing import Callable, Dict, List, Optional, Union

In [ ]:
#| hide
import os

import pandas as pd

from fastcore.test import test_close, test_fail
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#| hide
os.environ['NIXTLA_ID_AS_COL'] = '1'

# Accuracy Measurements

In [ ]:
#| exporti
def _metric_protections(y: np.ndarray, y_hat: np.ndarray, 
                        weights: Optional[np.ndarray]) -> None:
    if not ((weights is None) or (np.sum(weights) > 0)):
        raise Exception('Sum of `weights` cannot be 0')
    if not ((weights is None) or (weights.shape == y.shape)):
        raise Exception(
        f'Wrong weight dimension weights.shape {weights.shape}, y.shape {y.shape}')

def mse(y: np.ndarray, y_hat: np.ndarray, 
        weights: Optional[np.ndarray] = None,
        axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Mean Squared Error

    Calculates Mean Squared Error between
    `y` and `y_hat`. MSE measures the relative prediction
    accuracy of a forecasting method by calculating the 
    squared deviation of the prediction and the true
    value at a given time, and averages these devations
    over the length of the series.

    $$ \mathrm{MSE}(\\mathbf{y}_{\\tau}, \\mathbf{\hat{y}}_{\\tau}) = \\frac{1}{H} \\sum^{t+H}_{\\tau=t+1} (y_{\\tau} - \hat{y}_{\\tau})^{2} $$

    **Parameters:**<br>
    `y`: numpy array, Actual values.<br>
    `y_hat`: numpy array, Predicted values.<br>
    `mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>

    **Returns:**<br>
    `mse`: numpy array, (single value).
    """
    _metric_protections(y, y_hat, weights)

    delta_y = np.square(y - y_hat)
    if weights is not None:
        mse = np.average(delta_y[~np.isnan(delta_y)],
                         weights=weights[~np.isnan(delta_y)],
                         axis=axis)
    else:
        mse = np.nanmean(delta_y, axis=axis)
    return mse

def mqloss(y: np.ndarray, y_hat: np.ndarray, 
           quantiles: np.ndarray, 
           weights: Optional[np.ndarray] = None,
           axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Multi-Quantile Loss

    Calculates the Multi-Quantile loss (MQL) between `y` and `y_hat`.
    MQL calculates the average multi-quantile Loss for
    a given set of quantiles, based on the absolute 
    difference between predicted quantiles and observed values.

    $$ \mathrm{MQL}(\\mathbf{y}_{\\tau},[\\mathbf{\hat{y}}^{(q_{1})}_{\\tau}, ... ,\hat{y}^{(q_{n})}_{\\tau}]) = \\frac{1}{n} \\sum_{q_{i}} \mathrm{QL}(\\mathbf{y}_{\\tau}, \\mathbf{\hat{y}}^{(q_{i})}_{\\tau}) $$

    The limit behavior of MQL allows to measure the accuracy 
    of a full predictive distribution $\mathbf{\hat{F}}_{\\tau}$ with 
    the continuous ranked probability score (CRPS). This can be achieved 
    through a numerical integration technique, that discretizes the quantiles 
    and treats the CRPS integral with a left Riemann approximation, averaging over 
    uniformly distanced quantiles.    

    $$ \mathrm{CRPS}(y_{\\tau}, \mathbf{\hat{F}}_{\\tau}) = \int^{1}_{0} \mathrm{QL}(y_{\\tau}, \hat{y}^{(q)}_{\\tau}) dq $$

    **Parameters:**<br>
    `y`: numpy array, Actual values.<br>
    `y_hat`: numpy array, Predicted values.<br>
    `quantiles`: numpy array. Quantiles between 0 and 1, to perform evaluation upon size (n_quantiles).<br>
    `mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>
 
    **Returns:**<br>
    `mqloss`: numpy array, (single value).

    **References:**<br>
    [Roger Koenker and Gilbert Bassett, Jr., "Regression Quantiles".](https://www.jstor.org/stable/1913643)<br>
    [James E. Matheson and Robert L. Winkler, "Scoring Rules for Continuous Probability Distributions".](https://www.jstor.org/stable/2629907)
    """
    if weights is None: weights = np.ones(y.shape)
    if (np.sum(quantiles>1)>0 or np.sum(quantiles<0)>0):
        raise Exception('`quantiles` need to be between 0 and 1')
        
    _metric_protections(y, y_hat, weights)
    n_q = len(quantiles)
    
    y_rep  = np.expand_dims(y, axis=-1)
    error  = y_hat - y_rep
    sq     = np.maximum(-error, np.zeros_like(error))
    s1_q   = np.maximum(error, np.zeros_like(error))
    mqloss = (quantiles * sq + (1 - quantiles) * s1_q)
    
    # Match y/weights dimensions and compute weighted average
    weights = np.repeat(np.expand_dims(weights, axis=-1), repeats=n_q, axis=-1)
    mqloss  = np.average(mqloss, weights=weights, axis=axis)

    return mqloss

## Relative Mean Squared Error

In [ ]:
#| export
def rel_mse(y, y_hat, y_train, mask=None):
    """Relative Mean Squared Error

    Computes Relative mean squared error (RelMSE), as proposed by Hyndman & Koehler (2006)
    as an alternative to percentage errors, to avoid measure unstability.

    $$
    \mathrm{RelMSE}(\\mathbf{y}, \\mathbf{\hat{y}}, \\mathbf{\hat{y}}^{naive1}) =
    \\frac{\mathrm{MSE}(\\mathbf{y}, \\mathbf{\hat{y}})}{\mathrm{MSE}(\\mathbf{y}, \\mathbf{\hat{y}}^{naive1})}
    $$

    **Parameters:**<br>
    `y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
    `y_hat`: numpy array, Predicted values (`n_series`, `horizon`).<br>
    `mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>

    **Returns:**<br>
    `loss`: float.    

    **References:**<br>
    - [Hyndman, R. J and Koehler, A. B. (2006).
       "Another look at measures of forecast accuracy",
       International Journal of Forecasting, Volume 22, Issue 4.](https://www.sciencedirect.com/science/article/pii/S0169207006000239)<br>
    - [Kin G. Olivares, O. Nganba Meetei, Ruijun Ma, Rohan Reddy, Mengfei Cao, Lee Dicker. 
       "Probabilistic Hierarchical Forecasting with Deep Poisson Mixtures. 
       Submitted to the International Journal Forecasting, Working paper available at arxiv.](https://arxiv.org/pdf/2110.13179.pdf)
    """
    if mask is None: 
       mask = np.ones_like(y)
    n_series, horizon = y.shape

    eps = np.finfo(float).eps
    y_naive = np.repeat(y_train[:,[-1]], horizon, axis=1)
    norm = mse(y=y, y_hat=y_naive)
    loss = mse(y=y, y_hat=y_hat, weights=mask)
    loss = loss / (norm + eps)
    return loss

In [ ]:
show_doc(rel_mse)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/evaluation.py#L127){target="_blank" style="float:right; font-size:smaller"}

### rel_mse

>      rel_mse (y, y_hat, y_train, mask=None)

*Relative Mean Squared Error

Computes Relative mean squared error (RelMSE), as proposed by Hyndman & Koehler (2006)
as an alternative to percentage errors, to avoid measure unstability.

$$
\mathrm{RelMSE}(\mathbf{y}, \mathbf{\hat{y}}, \mathbf{\hat{y}}^{naive1}) =
\frac{\mathrm{MSE}(\mathbf{y}, \mathbf{\hat{y}})}{\mathrm{MSE}(\mathbf{y}, \mathbf{\hat{y}}^{naive1})}
$$

**Parameters:**<br>
`y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
`y_hat`: numpy array, Predicted values (`n_series`, `horizon`).<br>
`mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>

**Returns:**<br>
`loss`: float.    

**References:**<br>
- [Hyndman, R. J and Koehler, A. B. (2006).
   "Another look at measures of forecast accuracy",
   International Journal of Forecasting, Volume 22, Issue 4.](https://www.sciencedirect.com/science/article/pii/S0169207006000239)<br>
- [Kin G. Olivares, O. Nganba Meetei, Ruijun Ma, Rohan Reddy, Mengfei Cao, Lee Dicker. 
   "Probabilistic Hierarchical Forecasting with Deep Poisson Mixtures. 
   Submitted to the International Journal Forecasting, Working paper available at arxiv.](https://arxiv.org/pdf/2110.13179.pdf)*

## Mean Squared Scaled Error

In [ ]:
#| export
def msse(y, y_hat, y_train, mask=None):
    """Mean Squared Scaled Error

    Computes Mean squared scaled error (MSSE), as proposed by Hyndman & Koehler (2006)
    as an alternative to percentage errors, to avoid measure unstability.

    $$
    \\mathrm{MSSE}(\\mathbf{y}, \\mathbf{\\hat{y}}, \\mathbf{y}^{in-sample}) =
    \\frac{\\frac{1}{h} \\sum^{t+h}_{\\tau=t+1} (y_{\\tau} - \\hat{y}_{\\tau})^2}{\\frac{1}{t-1} \\sum^{t}_{\\tau=2} (y_{\\tau} - y_{\\tau-1})^2}
    $$

    where $n$ ($n=$`n`) is the size of the training data, and $h$ is the forecasting horizon ($h=$`horizon`).

    **Parameters:**<br>
    `y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
    `y_hat`: numpy array, Predicted values (`n_series`, `horizon`).<br>
    `y_train`: numpy array, Predicted values (`n_series`, `n`).<br>
    `mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>

    **Returns:**<br>
    `loss`: float.    

    **References:**<br>
    - [Hyndman, R. J and Koehler, A. B. (2006).
       "Another look at measures of forecast accuracy",
       International Journal of Forecasting, Volume 22, Issue 4.](https://www.sciencedirect.com/science/article/pii/S0169207006000239)<br>
    """
    if mask is None: 
       mask = np.ones_like(y)
    n_series, horizon = y.shape

    eps = np.finfo(float).eps
    y_in_sample_naive = y_train[:, :-1]
    y_in_sample_true = y_train[:, 1:]
    norm = mse(y=y_in_sample_true, y_hat=y_in_sample_naive)
    loss = mse(y=y, y_hat=y_hat, weights=mask)
    loss = loss / (norm + eps)
    return loss

In [ ]:
show_doc(msse)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/evaluation.py#L166){target="_blank" style="float:right; font-size:smaller"}

### msse

>      msse (y, y_hat, y_train, mask=None)

*Mean Squared Scaled Error

Computes Mean squared scaled error (MSSE), as proposed by Hyndman & Koehler (2006)
as an alternative to percentage errors, to avoid measure unstability.

$$
\mathrm{MSSE}(\mathbf{y}, \mathbf{\hat{y}}, \mathbf{y}^{in-sample}) =
\frac{\frac{1}{h} \sum^{t+h}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^2}{\frac{1}{t-1} \sum^{t}_{\tau=2} (y_{\tau} - y_{\tau-1})^2}
$$

where $n$ ($n=$`n`) is the size of the training data, and $h$ is the forecasting horizon ($h=$`horizon`).

**Parameters:**<br>
`y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
`y_hat`: numpy array, Predicted values (`n_series`, `horizon`).<br>
`y_train`: numpy array, Predicted values (`n_series`, `n`).<br>
`mask`: numpy array, Specifies date stamps per serie to consider in loss.<br>

**Returns:**<br>
`loss`: float.    

**References:**<br>
- [Hyndman, R. J and Koehler, A. B. (2006).
   "Another look at measures of forecast accuracy",
   International Journal of Forecasting, Volume 22, Issue 4.](https://www.sciencedirect.com/science/article/pii/S0169207006000239)<br>*

## Scaled CRPS

In [ ]:
#| export
def scaled_crps(y, y_hat, quantiles):
    """Scaled Continues Ranked Probability Score

    Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
    to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.

    This metric averages percentual weighted absolute deviations as 
    defined by the quantile losses.

    $$
    \mathrm{sCRPS}(\hat{F}_{\\tau}, \mathbf{y}_{\\tau}) = \\frac{2}{N} \sum_{i}
    \int^{1}_{0}
    \\frac{\mathrm{QL}(\hat{F}_{i,\\tau}, y_{i,\\tau})_{q}}{\sum_{i} | y_{i,\\tau} |} dq
    $$

    where $\hat{F}_{\\tau}$ is the an estimated multivariate distribution, and $y_{i,\\tau}$
    are its realizations.

    **Parameters:**<br>
    `y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
    `y_hat`: numpy array, Predicted quantiles of size (`n_series`, `horizon`, `n_quantiles`).<br>
    `quantiles`: numpy array,(`n_quantiles`). Quantiles to estimate from the distribution of y.<br>

    **Returns:**<br>
    `loss`: float.

    **References:**<br>
    - [Gneiting, Tilmann. (2011). \"Quantiles as optimal point forecasts\". 
    International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207010000063)<br>
    - [Spyros Makridakis, Evangelos Spiliotis, Vassilios Assimakopoulos, Zhi Chen, Anil Gaba, Ilia Tsetlin, Robert L. Winkler. (2022). 
    \"The M5 uncertainty competition: Results, findings and conclusions\". 
    International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207021001722)<br>
    - [Syama Sundar Rangapuram, Lucien D Werner, Konstantinos Benidis, Pedro Mercado, Jan Gasthaus, Tim Januschowski. (2021). 
    \"End-to-End Learning of Coherent Probabilistic Forecasts for Hierarchical Time Series\". 
    Proceedings of the 38th International Conference on Machine Learning (ICML).](https://proceedings.mlr.press/v139/rangapuram21a.html)
    """
    eps = np.finfo(float).eps
    norm  = np.sum(np.abs(y))
    loss  = mqloss(y=y, y_hat=y_hat, quantiles=quantiles)
    loss  = 2 * loss * np.sum(np.ones(y.shape)) / (norm + eps)
    return loss

In [ ]:
show_doc(scaled_crps)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/evaluation.py#L206){target="_blank" style="float:right; font-size:smaller"}

### scaled_crps

>      scaled_crps (y, y_hat, quantiles)

*Scaled Continues Ranked Probability Score

Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.

This metric averages percentual weighted absolute deviations as 
defined by the quantile losses.

$$
\mathrm{sCRPS}(\hat{F}_{\tau}, \mathbf{y}_{\tau}) = \frac{2}{N} \sum_{i}
\int^{1}_{0}
\frac{\mathrm{QL}(\hat{F}_{i,\tau}, y_{i,\tau})_{q}}{\sum_{i} | y_{i,\tau} |} dq
$$

where $\hat{F}_{\tau}$ is the an estimated multivariate distribution, and $y_{i,\tau}$
are its realizations.

**Parameters:**<br>
`y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
`y_hat`: numpy array, Predicted quantiles of size (`n_series`, `horizon`, `n_quantiles`).<br>
`quantiles`: numpy array,(`n_quantiles`). Quantiles to estimate from the distribution of y.<br>

**Returns:**<br>
`loss`: float.

**References:**<br>
- [Gneiting, Tilmann. (2011). "Quantiles as optimal point forecasts". 
International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207010000063)<br>
- [Spyros Makridakis, Evangelos Spiliotis, Vassilios Assimakopoulos, Zhi Chen, Anil Gaba, Ilia Tsetlin, Robert L. Winkler. (2022). 
"The M5 uncertainty competition: Results, findings and conclusions". 
International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207021001722)<br>
- [Syama Sundar Rangapuram, Lucien D Werner, Konstantinos Benidis, Pedro Mercado, Jan Gasthaus, Tim Januschowski. (2021). 
"End-to-End Learning of Coherent Probabilistic Forecasts for Hierarchical Time Series". 
Proceedings of the 38th International Conference on Machine Learning (ICML).](https://proceedings.mlr.press/v139/rangapuram21a.html)*

## Energy Score

In [ ]:
#| export
def energy_score(y, y_sample1, y_sample2, beta=2):
    """Energy Score

    Calculates Gneiting's Energy Score sample approximation for
    `y` and independent multivariate samples `y_sample1` and `y_sample2`.
    The Energy Score generalizes the CRPS (`beta`=1) in the multivariate setting.

    $$
    \mathrm{ES}(\\mathbf{y}_{\\tau}, \\mathbf{\hat{y}}_{\\tau}, \\mathbf{\hat{y}}_{\\tau}') 
    = \\frac{1}{2} \mathbb{E}_{\hat{P}} \\left[ ||\\mathbf{\hat{y}}_{\\tau} - \\mathbf{\hat{y}}_{\\tau}'||^{\\beta} \\right]
    -  \mathbb{E}_{\hat{P}} \\left[ ||\\mathbf{y}_{\\tau} - \\mathbf{\hat{y}}_{\\tau}||^{\\beta} \\right] 
    \quad \\beta \in (0,2]
    $$

    where $\\mathbf{\hat{y}}_{\\tau}, \\mathbf{\hat{y}}_{\\tau}'$ are independent samples drawn from $\hat{P}$.

    **Parameters:**<br>
    `y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
    `y_sample1`: numpy array, predictive distribution sample of size (`n_series`, `horizon`, `n_samples`).<br>
    `y_sample2`: numpy array, predictive distribution sample of size (`n_series`, `horizon`, `n_samples`).<br>
    `beta`: float in (0,2], defines the energy score's power for the euclidean metric.<br>

    **Returns:**<br>
    `score`: float.

    **References:**<br>
    - [Gneiting, Tilmann, and Adrian E. Raftery. (2007). 
    \"Strictly proper scoring rules, prediction and estimation\". 
    Journal of the American Statistical Association.](https://sites.stat.washington.edu/raftery/Research/PDF/Gneiting2007jasa.pdf)<br>
    - [Anastasios Panagiotelis, Puwasala Gamakumara, George Athanasopoulos, Rob J. Hyndman. (2022). 
    \"Probabilistic forecast reconciliation: Properties, evaluation and score optimisation\". 
    European Journal of Operational Research.](https://www.sciencedirect.com/science/article/pii/S0377221722006087)    
    """
    if beta>2 or beta<0:
        raise Exception("beta needs to be between 0 and 2.")

    dif1 = (y_sample1 - y_sample2)
    dif2 = (y[:,:,None] - y_sample1)

    term1 = np.linalg.norm(dif1, axis=0) ** beta
    term2 = np.linalg.norm(dif2, axis=0) ** beta

    score = np.mean(term2 - 0.5 * term1)
    return score

In [ ]:
show_doc(energy_score)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/evaluation.py#L249){target="_blank" style="float:right; font-size:smaller"}

### energy_score

>      energy_score (y, y_sample1, y_sample2, beta=2)

*Energy Score

Calculates Gneiting's Energy Score sample approximation for
`y` and independent multivariate samples `y_sample1` and `y_sample2`.
The Energy Score generalizes the CRPS (`beta`=1) in the multivariate setting.

$$
\mathrm{ES}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}_{\tau}') 
= \frac{1}{2} \mathbb{E}_{\hat{P}} \left[ ||\mathbf{\hat{y}}_{\tau} - \mathbf{\hat{y}}_{\tau}'||^{\beta} \right]
-  \mathbb{E}_{\hat{P}} \left[ ||\mathbf{y}_{\tau} - \mathbf{\hat{y}}_{\tau}||^{\beta} \right] 
\quad \beta \in (0,2]
$$

where $\mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}_{\tau}'$ are independent samples drawn from $\hat{P}$.

**Parameters:**<br>
`y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
`y_sample1`: numpy array, predictive distribution sample of size (`n_series`, `horizon`, `n_samples`).<br>
`y_sample2`: numpy array, predictive distribution sample of size (`n_series`, `horizon`, `n_samples`).<br>
`beta`: float in (0,2], defines the energy score's power for the euclidean metric.<br>

**Returns:**<br>
`score`: float.

**References:**<br>
- [Gneiting, Tilmann, and Adrian E. Raftery. (2007). 
"Strictly proper scoring rules, prediction and estimation". 
Journal of the American Statistical Association.](https://sites.stat.washington.edu/raftery/Research/PDF/Gneiting2007jasa.pdf)<br>
- [Anastasios Panagiotelis, Puwasala Gamakumara, George Athanasopoulos, Rob J. Hyndman. (2022). 
"Probabilistic forecast reconciliation: Properties, evaluation and score optimisation". 
European Journal of Operational Research.](https://www.sciencedirect.com/science/article/pii/S0377221722006087)*

In [ ]:
#| export
def log_score(y, y_hat, cov, allow_singular=True):
    """ Log Score.

    One of the simplest multivariate probability scoring rules,
    it evaluates the negative density at the value of the realisation.

    $$
    \mathrm{LS}(\\mathbf{y}_{\\tau}, \\mathbf{P}(\\theta_{\\tau}))
    = - \\log(f(\\mathbf{y}_{\\tau}, \\theta_{\\tau}))
    $$

    where $f$ is the density, $\\mathbf{P}(\\theta_{\\tau})$ is a 
    parametric distribution and $f(\\mathbf{y}_{\\tau}, \\theta_{\\tau})$
    represents its density. 
    For the moment we only support multivariate normal log score.

    $$
    f(\\mathbf{y}_{\\tau}, \\theta_{\\tau}) =
    (2\\pi )^{-k/2}\\det({\\boldsymbol{\Sigma }})^{-1/2}
    \,\\exp \\left(
    -{\\frac {1}{2}}(\mathbf{y}_{\\tau} -\\hat{\mathbf{y}}_{\\tau})^{\!{\mathsf{T}}}
    {\\boldsymbol{\Sigma }}^{-1}
    (\mathbf{y}_{\\tau} -\\hat{\mathbf{y}}_{\\tau})
    \\right)
    $$

    **Parameters:**<br>
    `y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
    `y_hat`: numpy array, Predicted values (`n_series`, `horizon`).<br>
    `cov`: numpy matrix, Predicted values covariance (`n_series`, `n_series`, `horizon`).<br>
    `allow_singular`: bool=True, if true allows singular covariance.<br>

    **Returns:**<br>
    `score`: float.
    """
    scores = [multivariate_normal.pdf(x=y[:,h], 
                  mean=y_hat[:,h], cov=cov[:,:,h], allow_singular=allow_singular) \
                  for h in range(y.shape[1])]
    score = np.mean(scores)
    return score

In [ ]:
show_doc(log_score)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/evaluation.py#L295){target="_blank" style="float:right; font-size:smaller"}

### log_score

>      log_score (y, y_hat, cov, allow_singular=True)

*Log Score.

One of the simplest multivariate probability scoring rules,
it evaluates the negative density at the value of the realisation.

$$
\mathrm{LS}(\mathbf{y}_{\tau}, \mathbf{P}(\theta_{\tau}))
= - \log(f(\mathbf{y}_{\tau}, \theta_{\tau}))
$$

where $f$ is the density, $\mathbf{P}(\theta_{\tau})$ is a 
parametric distribution and $f(\mathbf{y}_{\tau}, \theta_{\tau})$
represents its density. 
For the moment we only support multivariate normal log score.

$$
f(\mathbf{y}_{\tau}, \theta_{\tau}) =
(2\pi )^{-k/2}\det({\boldsymbol{\Sigma }})^{-1/2}
\,\exp \left(
-{\frac {1}{2}}(\mathbf{y}_{\tau} -\hat{\mathbf{y}}_{\tau})^{\!{\mathsf{T}}}
{\boldsymbol{\Sigma }}^{-1}
(\mathbf{y}_{\tau} -\hat{\mathbf{y}}_{\tau})
\right)
$$

**Parameters:**<br>
`y`: numpy array, Actual values of size (`n_series`, `horizon`).<br>
`y_hat`: numpy array, Predicted values (`n_series`, `horizon`).<br>
`cov`: numpy matrix, Predicted values covariance (`n_series`, `n_series`, `horizon`).<br>
`allow_singular`: bool=True, if true allows singular covariance.<br>

**Returns:**<br>
`score`: float.*

In [ ]:
x = np.linspace(0, 5, 10, endpoint=False)
y = multivariate_normal.pdf(x, mean=2.5, cov=0.5)
y

array([0.00108914, 0.01033349, 0.05946514, 0.20755375, 0.43939129,
       0.56418958, 0.43939129, 0.20755375, 0.05946514, 0.01033349])

# Hierarchical Evaluation

In [ ]:
#| export
class HierarchicalEvaluation:
    """Hierarchical Evaluation Class.
    
    You can use your own metrics to evaluate the performance of each level in the structure.
    The metrics receive `y` and `y_hat` as arguments and they are numpy arrays of size `(series, horizon)`.
    Consider, for example, the function `rmse` that calculates the root mean squared error.

    This class facilitates measurements across the hierarchy, defined by the `tags` list.
    See also the [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).

    **Parameters:**<br>
    `evaluators`: functions with arguments `y`, `y_hat` (numpy arrays).<br>

    **References:**<br>
    """
    def __init__(self, 
                 evaluators: List[Callable]):
        self.evaluators = evaluators

    def evaluate(self, 
                 Y_hat_df: Frame,
                 Y_test_df: Frame,
                 tags: Dict[str, np.ndarray],
                 Y_df: Optional[Frame] = None,
                 benchmark: Optional[str] = None,
                 id_col: str = "unique_id",
                 time_col: str = "ds", 
                 target_col: str = "y", 
                 ):
        """Hierarchical Evaluation Method.

        **Parameters:**<br>
        `Y_hat_df`: DataFrame, Forecasts with columns `'unique_id'`, `'ds'` and models to evaluate.<br>
        `Y_test_df`:  DataFrame, Observed values with columns `['unique_id', 'ds', 'y']`.<br>
        `tags`: np.array, each str key is a level and its value contains tags associated to that level.<br>
        `Y_df`: DataFrame, Training set of base time series with columns `['unique_id', 'ds', 'y']`.<br>
        `benchmark`: str, If passed, evaluators are scaled by the error of this benchark.<br>
        `id_col` : str='unique_id', column that identifies each serie.<br>
        `time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
        `target_col` : str='y', column that contains the target.<br>

        **Returns:**<br>
        `evaluation`: DataFrame with accuracy measurements across hierarchical levels.
        """
        Y_hat_nw = _to_narwhals_maybe_warn_and_reset_idx(Y_hat_df, id_col)
        Y_test_nw = _to_narwhals_maybe_warn_and_reset_idx(Y_test_df, id_col)
        native_namespace = nw.get_native_namespace(Y_hat_nw)
        if Y_df is not None:
            Y_nw = _to_narwhals_maybe_warn_and_reset_idx(Y_df, id_col)

        n_series = len(set(Y_hat_nw[id_col]))
        h = len(set(Y_hat_nw[time_col]))
        if len(Y_hat_nw) != n_series * h:
            raise Exception('Y_hat_df should have a forecast for each series and horizon')

        fn_names = [fn.__name__ for fn in self.evaluators]
        has_y_insample = any(['y_insample' in signature(fn).parameters for fn in self.evaluators])
        if has_y_insample and Y_df is None:
            raise Exception('At least one evaluator needs y_insample, please pass `Y_df`')

        if benchmark is not None:
            fn_names = [f'{fn_name}-scaled' for fn_name in fn_names]

        tags_ = {'Overall': np.concatenate(list(tags.values()))}
        tags_ = {**tags, **tags_}

        model_names = [c for c in Y_hat_nw.columns if c not in [id_col, time_col, target_col]]
        evaluation_np = np.empty((len(tags_), len(fn_names), len(model_names)), dtype=np.float64)
        evaluation_index_np = np.empty((len(tags_) * len(fn_names), 2), dtype=object)
        Y_h = Y_hat_nw.join(Y_test_nw, how="left", on=[id_col, time_col]).sort(by=[id_col, time_col])
        for i_level, (level, cats) in enumerate(tags_.items()):
            Y_h_cats = Y_h.filter(nw.col(id_col).is_in(cats))
            y_test_cats = Y_h_cats[target_col].to_numpy().reshape(-1, h)

            if has_y_insample and Y_df is not None:
                y_insample = Y_nw.pivot(on=time_col, index = id_col, values = target_col, sort_columns=True).sort(by=id_col)
                y_insample_cols_ex_id_col = y_insample.columns
                y_insample_cols_ex_id_col.remove(id_col)
                y_insample = y_insample.filter(nw.col(id_col).is_in(cats))
                y_insample = y_insample.select(nw.col(y_insample_cols_ex_id_col)).to_numpy()

            for i_fn, fn in enumerate(self.evaluators):
                if 'y_insample' in signature(fn).parameters:
                    kwargs = {'y_insample': y_insample}
                else:
                    kwargs = {}
                fn_name = fn_names[i_fn]
                for i_model, model in enumerate(model_names):
                    loss = fn(y_test_cats, Y_h_cats[model].to_numpy().reshape(-1, h), **kwargs)
                    if benchmark is not None:
                        scale = fn(y_test_cats, Y_h_cats[benchmark].to_numpy().reshape(-1, h), **kwargs)
                        if np.isclose(scale, 0., atol=np.finfo(float).eps):
                            scale += np.finfo(float).eps
                            if np.isclose(scale, loss, atol=1e-8):
                                scale = 1.
                        loss /= scale

                    evaluation_np[i_level, i_fn, i_model] = loss
                    evaluation_index_np[i_level * len(fn_names) + i_fn, 0] = level
                    evaluation_index_np[i_level * len(fn_names) + i_fn, 1] = fn_name

        evaluation_np = evaluation_np.reshape(-1, len(model_names))
        evaluation_index_dict = {"level": evaluation_index_np[:, 0], "metric": evaluation_index_np[:, 1]}
        evaluation_index_nw = nw.from_dict(evaluation_index_dict, native_namespace=native_namespace)
        evaluation_dict = dict(zip(model_names, evaluation_np.T))
        evaluation_nw = evaluation_index_nw.with_columns(**evaluation_dict)
        evaluation_nw = evaluation_nw[["level", "metric"] + model_names]

        evaluation = _to_native_maybe_set_index(evaluation_nw, ["level", "metric"])

        return evaluation

In [ ]:
show_doc(HierarchicalEvaluation, title_level=3)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/evaluation.py#L340){target="_blank" style="float:right; font-size:smaller"}

### HierarchicalEvaluation

>      HierarchicalEvaluation (evaluators:List[Callable])

*Hierarchical Evaluation Class.

You can use your own metrics to evaluate the performance of each level in the structure.
The metrics receive `y` and `y_hat` as arguments and they are numpy arrays of size `(series, horizon)`.
Consider, for example, the function `rmse` that calculates the root mean squared error.

This class facilitates measurements across the hierarchy, defined by the `tags` list.
See also the [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).

**Parameters:**<br>
`evaluators`: functions with arguments `y`, `y_hat` (numpy arrays).<br>

**References:**<br>*

In [ ]:
show_doc(HierarchicalEvaluation.evaluate, title_level=3)

---

[source](https://github.com/Nixtla/hierarchicalforecast/blob/main/hierarchicalforecast/evaluation.py#L359){target="_blank" style="float:right; font-size:smaller"}

### HierarchicalEvaluation.evaluate

>      HierarchicalEvaluation.evaluate (Y_hat_df:Union[ForwardRef('DataFrame[Any
>                                       ]'),ForwardRef('LazyFrame[Any]')], Y_tes
>                                       t_df:Union[ForwardRef('DataFrame[Any]'),
>                                       ForwardRef('LazyFrame[Any]')],
>                                       tags:Dict[str,numpy.ndarray], Y_df:Union
>                                       [ForwardRef('DataFrame[Any]'),ForwardRef
>                                       ('LazyFrame[Any]'),NoneType]=None,
>                                       benchmark:Optional[str]=None,
>                                       id_col:str='unique_id',
>                                       time_col:str='ds', target_col:str='y')

*Hierarchical Evaluation Method.

**Parameters:**<br>
`Y_hat_df`: DataFrame, Forecasts with columns `'unique_id'`, `'ds'` and models to evaluate.<br>
`Y_test_df`:  DataFrame, Observed values with columns `['unique_id', 'ds', 'y']`.<br>
`tags`: np.array, each str key is a level and its value contains tags associated to that level.<br>
`Y_df`: DataFrame, Training set of base time series with columns `['unique_id', 'ds', 'y']`.<br>
`benchmark`: str, If passed, evaluators are scaled by the error of this benchark.<br>
`id_col` : str='unique_id', column that identifies each serie.<br>
`time_col` : str='ds', column that identifies each timestep, its values can be timestamps or integers.<br>
`target_col` : str='y', column that contains the target.<br>

**Returns:**<br>
`evaluation`: DataFrame with accuracy measurements across hierarchical levels.*

# Example

In [ ]:
#| hide
def rmse(y, y_hat):
    return np.mean(np.sqrt(np.mean((y-y_hat)**2, axis=1)))

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

In [ ]:
#| hide
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace, ERM
from hierarchicalforecast.utils import aggregate

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# non strictly hierarchical structure
hiers_grouped = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]
# strictly hierarchical structure
hiers_strictly = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'State', 'Region'], 
]

# getting df
hier_grouped_df, S_grouped, tags_grouped = aggregate(df, hiers_grouped)

#split train/test
hier_grouped_df['y_model'] = hier_grouped_df['y']
# we should be able to recover y using the methods
hier_grouped_df_h = hier_grouped_df.groupby('unique_id').tail(12).reset_index(drop=True)
ds_h = hier_grouped_df_h['ds'].unique()
hier_grouped_df = hier_grouped_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
rng = np.random.default_rng(0)
hier_grouped_df['y_model'] += rng.uniform(-1, 1, len(hier_grouped_df))

#hierachical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    # ERM recovers but needs bigger eps
    # ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_df_h, Y_df=hier_grouped_df, 
                            S=S_grouped, tags=tags_grouped)

In [ ]:
#| hide
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)
def rmse(y, y_hat):
    return np.sqrt(mse(y, y_hat))

evaluator = HierarchicalEvaluation([mse, rmse])
evaluator.evaluate(Y_hat_df=reconciled.drop(columns='y'), 
                   Y_test_df=reconciled[['unique_id', 'ds', 'y']], 
                   tags=tags_grouped,
                #    benchmark='y_model',
                   )

,level,metric,y_model,y_model/BottomUp,y_model/MinTrace_method-ols,y_model/MinTrace_method-wls_struct,y_model/MinTrace_method-wls_var,y_model/MinTrace_method-mint_shrink
0,Country,mse,0.0,5.514537e-23,5.514537e-23,8.271806e-23,3.970467e-23,3.749885e-23
1,Country,rmse,0.0,7.425993e-12,7.425993e-12,9.094947e-12,6.301164e-12,6.123631e-12
2,Country/State,mse,0.0,1.067575e-24,1.316939e-24,1.185412e-24,1.067979e-24,1.567969e-24
3,Country/State,rmse,0.0,1.033235e-12,1.147580e-12,1.088766e-12,1.033431e-12,1.252186e-12
4,Country/Purpose,mse,0.0,1.280622e-24,3.682462e-24,1.816997e-24,2.081953e-24,1.692058e-24
5,Country/Purpose,rmse,0.0,1.131646e-12,1.918974e-12,1.347960e-12,1.442898e-12,1.300791e-12
6,Country/State/Region,mse,0.0,2.424395e-27,5.732605e-26,2.453493e-26,3.601985e-26,2.967922e-26
7,Country/State/Region,rmse,0.0,4.923815e-14,2.394286e-13,1.566363e-13,1.897890e-13,1.722766e-13
8,Country/State/Purpose,mse,0.0,2.003747e-26,2.074433e-25,1.004139e-25,1.028776e-25,1.034770e-25
9,Country/State/Purpose,rmse,0.0,1.415538e-13,4.554595e-13,3.168816e-13,3.207454e-13,3.216784e-13


In [ ]:
#| hide
# polars
import polars as pl

In [ ]:
#| hide
# polars
reconciled_pl = pl.from_pandas(reconciled)
evaluator.evaluate(Y_hat_df=reconciled_pl.drop('y'), 
                   Y_test_df=reconciled_pl[['unique_id', 'ds', 'y']], 
                   tags=tags_grouped,
                   benchmark='y_model')

level,metric,y_model,y_model/BottomUp,y_model/MinTrace_method-ols,y_model/MinTrace_method-wls_struct,y_model/MinTrace_method-wls_var,y_model/MinTrace_method-mint_shrink
str,str,f64,f64,f64,f64,f64,f64
"""Country""","""mse-scaled""",0.0,5.5145e-23,5.5145e-23,8.2718e-23,3.9705e-23,3.7499e-23
"""Country""","""rmse-scaled""",0.0,7.4260e-12,7.4260e-12,9.0949e-12,6.3012e-12,6.1236e-12
"""Country/State""","""mse-scaled""",0.0,1.0676e-24,1.3169e-24,1.1854e-24,1.0680e-24,1.5680e-24
"""Country/State""","""rmse-scaled""",0.0,1.0332e-12,1.1476e-12,1.0888e-12,1.0334e-12,1.2522e-12
"""Country/Purpose""","""mse-scaled""",0.0,1.2806e-24,3.6825e-24,1.8170e-24,2.0820e-24,1.6921e-24
…,…,…,…,…,…,…,…
"""Country/State/Purpose""","""rmse-scaled""",0.0,1.4155e-13,4.5546e-13,3.1688e-13,3.2075e-13,3.2168e-13
"""Country/State/Region/Purpose""","""mse-scaled""",0.0,0.0,8.8109e-27,4.2567e-27,8.1267e-27,7.7364e-27
"""Country/State/Region/Purpose""","""rmse-scaled""",0.0,0.0,9.3866e-14,6.5243e-14,9.0148e-14,8.7957e-14


In [ ]:
#| hide
def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

evaluator = HierarchicalEvaluation([mase])
evaluator.evaluate(Y_hat_df=reconciled.drop(columns='y'), 
                   Y_test_df=reconciled[['unique_id', 'ds', 'y']], 
                   tags=tags_grouped,
                   Y_df=hier_grouped_df,
                   benchmark='y_model')

,level,metric,y_model,y_model/BottomUp,y_model/MinTrace_method-ols,y_model/MinTrace_method-wls_struct,y_model/MinTrace_method-wls_var,y_model/MinTrace_method-mint_shrink
0,Country,mase-scaled,0.0,5.953168e-15,5.953168e-15,8.268289e-15,5.953168e-15,5.291705e-15
1,Country/State,mase-scaled,0.0,1.980839e-15,4.496669e-15,3.618865e-15,3.551700e-15,3.914717e-15
2,Country/Purpose,mase-scaled,0.0,2.015436e-15,3.932052e-15,3.050845e-15,3.368266e-15,2.825466e-15
3,Country/State/Region,mase-scaled,0.0,2.757523e-16,3.934661e-15,2.423732e-15,3.245747e-15,3.240262e-15
4,Country/State/Purpose,mase-scaled,0.0,5.247942e-16,4.600730e-15,2.784470e-15,3.505178e-15,3.788154e-15
5,Country/State/Region/Purpose,mase-scaled,0.0,0.000000e+00,4.048205e-15,2.349369e-15,4.152554e-15,4.260422e-15
6,Overall,mase-scaled,0.0,1.590876e-16,4.081333e-15,2.439853e-15,3.927197e-15,4.024848e-15


In [ ]:
#| hide
# polars
reconciled_pl = pl.from_pandas(reconciled)
hier_grouped_df_pl = pl.from_pandas(hier_grouped_df)

evaluator.evaluate(Y_hat_df=reconciled_pl.drop('y'), 
                   Y_test_df=reconciled_pl[['unique_id', 'ds', 'y']], 
                   tags=tags_grouped,
                   Y_df=hier_grouped_df_pl,
                   benchmark='y_model')

level,metric,y_model,y_model/BottomUp,y_model/MinTrace_method-ols,y_model/MinTrace_method-wls_struct,y_model/MinTrace_method-wls_var,y_model/MinTrace_method-mint_shrink
str,str,f64,f64,f64,f64,f64,f64
"""Country""","""mase-scaled""",0.0,5.9532e-15,5.9532e-15,8.2683e-15,5.9532e-15,5.2917e-15
"""Country/State""","""mase-scaled""",0.0,1.9808e-15,4.4967e-15,3.6189e-15,3.5517e-15,3.9147e-15
"""Country/Purpose""","""mase-scaled""",0.0,2.0154e-15,3.9321e-15,3.0508e-15,3.3683e-15,2.8255e-15
"""Country/State/Region""","""mase-scaled""",0.0,2.7575e-16,3.9347e-15,2.4237e-15,3.2457e-15,3.2403e-15
"""Country/State/Purpose""","""mase-scaled""",0.0,5.2479e-16,4.6007e-15,2.7845e-15,3.5052e-15,3.7882e-15
"""Country/State/Region/Purpose""","""mase-scaled""",0.0,0.0,4.0482e-15,2.3494e-15,4.1526e-15,4.2604e-15
"""Overall""","""mase-scaled""",0.0,1.5909e-16,4.0813e-15,2.4399e-15,3.9272e-15,4.0248e-15


In [ ]:
#| hide
# test work for h=1
evaluator = HierarchicalEvaluation([mase])
evaluator.evaluate(Y_hat_df=reconciled.groupby('unique_id').tail(1).drop(columns='y'), 
                   Y_test_df=reconciled.groupby('unique_id').tail(1)[['unique_id', 'ds', 'y']], 
                   tags=tags_grouped,
                   Y_df=hier_grouped_df,
                   benchmark='y_model')

,level,metric,y_model,y_model/BottomUp,y_model/MinTrace_method-ols,y_model/MinTrace_method-wls_struct,y_model/MinTrace_method-wls_var,y_model/MinTrace_method-mint_shrink
0,Country,mase-scaled,0.0,1.190634e-14,3.968779e-15,1.190634e-14,7.937557e-15,3.968779e-15
1,Country/State,mase-scaled,0.0,1.937913e-15,4.988451e-15,2.785879e-15,3.790787e-15,3.077111e-15
2,Country/Purpose,mase-scaled,0.0,4.812372e-16,5.812527e-15,1.246659e-15,4.596299e-15,2.190113e-15
3,Country/State/Region,mase-scaled,0.0,2.531390e-16,3.806263e-15,1.876343e-15,3.213134e-15,3.428183e-15
4,Country/State/Purpose,mase-scaled,0.0,4.054006e-16,5.302327e-15,2.481178e-15,4.302359e-15,3.255292e-15
5,Country/State/Region/Purpose,mase-scaled,0.0,0.000000e+00,4.312915e-15,2.098706e-15,4.913715e-15,4.313319e-15
6,Overall,mase-scaled,0.0,1.448140e-16,4.322831e-15,2.115733e-15,4.546570e-15,4.031309e-15


In [ ]:
#| hide
# polars
# test work for h=1
evaluator.evaluate(Y_hat_df=reconciled_pl.group_by('unique_id').tail(1).drop('y'), 
                   Y_test_df=reconciled_pl.group_by('unique_id').tail(1)[['unique_id', 'ds', 'y']], 
                   tags=tags_grouped,
                   Y_df=hier_grouped_df_pl,
                   benchmark='y_model')

level,metric,y_model,y_model/BottomUp,y_model/MinTrace_method-ols,y_model/MinTrace_method-wls_struct,y_model/MinTrace_method-wls_var,y_model/MinTrace_method-mint_shrink
str,str,f64,f64,f64,f64,f64,f64
"""Country""","""mase-scaled""",0.0,1.1906e-14,3.9688e-15,1.1906e-14,7.9376e-15,3.9688e-15
"""Country/State""","""mase-scaled""",0.0,1.9379e-15,4.9885e-15,2.7859e-15,3.7908e-15,3.0771e-15
"""Country/Purpose""","""mase-scaled""",0.0,4.8124e-16,5.8125e-15,1.2467e-15,4.5963e-15,2.1901e-15
"""Country/State/Region""","""mase-scaled""",0.0,2.5314e-16,3.8063e-15,1.8763e-15,3.2131e-15,3.4282e-15
"""Country/State/Purpose""","""mase-scaled""",0.0,4.0540e-16,5.3023e-15,2.4812e-15,4.3024e-15,3.2553e-15
"""Country/State/Region/Purpose""","""mase-scaled""",0.0,0.0,4.3129e-15,2.0987e-15,4.9137e-15,4.3133e-15
"""Overall""","""mase-scaled""",0.0,1.4481e-16,4.3228e-15,2.1157e-15,4.5466e-15,4.0313e-15


In [ ]:
#| hide
from statsforecast.core import StatsForecast
from statsforecast.models import AutoETS

In [ ]:
#| hide

# Load TourismSmall dataset
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')
qs = df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
df['ds'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()

# Create hierarchical seires based on geographic levels and purpose
# And Convert quarterly ds string to pd.datetime format
hierarchy_levels = [['Country'],
                    ['Country', 'State'], 
                    ['Country', 'Purpose'], 
                    ['Country', 'State', 'Region'], 
                    ['Country', 'State', 'Purpose'], 
                    ['Country', 'State', 'Region', 'Purpose']]

Y_df, S_df, tags = aggregate(df=df, spec=hierarchy_levels)

# Split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

# Compute base auto-ETS predictions
# Careful identifying correct data freq, this data quarterly 'Q'
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], freq='QS', n_jobs=-1)
Y_hat_df = fcst.forecast(df=Y_train_df, h=8, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

def rmse(y, y_hat):
    return np.mean(np.sqrt(np.mean((y-y_hat)**2, axis=1)))

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

reconcilers = [
                BottomUp(),
                MinTrace(method='ols'),
                MinTrace(method='mint_shrink'),
               ]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)

# Evaluate
eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']

evaluator = HierarchicalEvaluation(evaluators=[mase, rmse])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

evaluation_check = pd.DataFrame({
    'level': ['Total', 'Purpose', 'State', 'Regions', 'Bottom', 'Overall'],
    'metric': 6 * ['mase'],
    'AutoETS': [1.59, 1.32, 1.39, 1.12, 0.98, 1.02],
    'AutoETS/BottomUp': [3.16, 2.28, 1.90, 1.19, 0.98, 1.06],
})

pd.testing.assert_frame_equal(evaluation.query("metric == 'mase'")[["level", "metric", "AutoETS", "AutoETS/BottomUp"]].reset_index(drop=True), evaluation_check)

In [ ]:
#| hide
# polars

# Test if polars gives equal results to pandas
df_pl = pl.from_pandas(df)
Y_df, S_df, tags = aggregate(df=df_pl, spec=hierarchy_levels)

# Split train/test sets
Y_test_df  = Y_df.group_by('unique_id').tail(8)
Y_train_df = Y_df.group_by('unique_id').head(72)

# Compute base auto-ETS predictions
# Careful identifying correct data freq, this data quarterly 'Q'
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], freq='1q', n_jobs=-1)
Y_hat_df_pl = fcst.forecast(df=Y_train_df, h=8, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

# Reconcile the base predictions
reconcilers = [BottomUp(),
                MinTrace(method='ols'),               
               MinTrace(method='mint_shrink')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df_pl = hrec.reconcile(Y_hat_df=Y_hat_df_pl, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)

# Evaluate
evaluator = HierarchicalEvaluation(evaluators=[mase, rmse])
evaluation_pl = evaluator.evaluate(
        Y_hat_df=Y_rec_df_pl, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)
evaluation_pl = evaluation_pl.with_columns(pl.col(numeric_cols).round(2).cast(pl.Float64))

# Check if polars gives identical results as pandas
pd.testing.assert_frame_equal(Y_hat_df_pl.to_pandas(), Y_hat_df)
pd.testing.assert_frame_equal(Y_rec_df_pl.to_pandas(), Y_rec_df)
pd.testing.assert_frame_equal(evaluation_pl.to_pandas(), evaluation)

# References

- [Gneiting, Tilmann, and Adrian E. Raftery. (2007). 
\"Strictly proper scoring rules, prediction and estimation\". 
Journal of the American Statistical Association.](https://sites.stat.washington.edu/raftery/Research/PDF/Gneiting2007jasa.pdf)
- [Gneiting, Tilmann. (2011). \"Quantiles as optimal point forecasts\". 
International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207010000063)
- [Spyros Makridakis, Evangelos Spiliotis, Vassilios Assimakopoulos, Zhi Chen, Anil Gaba, Ilia Tsetlin, Robert L. Winkler. (2022). 
\"The M5 uncertainty competition: Results, findings and conclusions\". 
International Journal of Forecasting.](https://www.sciencedirect.com/science/article/pii/S0169207021001722)
- [Anastasios Panagiotelis, Puwasala Gamakumara, George Athanasopoulos, Rob J. Hyndman. (2022). 
\"Probabilistic forecast reconciliation: Properties, evaluation and score optimisation\". 
European Journal of Operational Research.](https://www.sciencedirect.com/science/article/pii/S0377221722006087)
- [Syama Sundar Rangapuram, Lucien D Werner, Konstantinos Benidis, Pedro Mercado, Jan Gasthaus, Tim Januschowski. (2021). 
\"End-to-End Learning of Coherent Probabilistic Forecasts for Hierarchical Time Series\". 
Proceedings of the 38th International Conference on Machine Learning (ICML).](https://proceedings.mlr.press/v139/rangapuram21a.html)
- [Kin G. Olivares, O. Nganba Meetei, Ruijun Ma, Rohan Reddy, Mengfei Cao, Lee Dicker (2022). 
"Probabilistic Hierarchical Forecasting with Deep Poisson Mixtures". 
Submitted to the International Journal Forecasting, Working paper available at arxiv.](https://arxiv.org/pdf/2110.13179.pdf)
- [Makridakis, S., Spiliotis E., and Assimakopoulos V. (2022). 
"M5 Accuracy Competition: Results, Findings, and Conclusions.",
International Journal of Forecasting, Volume 38, Issue 4.](https://www.sciencedirect.com/science/article/pii/S0169207021001874)